In [1]:
import sys
sys.path.append('../cloud_classifier')

import cloud_trainer
import data_handler
import importlib
importlib.reload(cloud_trainer)
importlib.reload(data_handler)

ct = cloud_trainer.cloud_trainer()
dh = data_handler.data_handler()

#### Read all sets from folder

In [2]:
import os
import re
import numpy as np

#TODO: os.path.join().
dh.training_sets = None
folder = "../data/cloud-typing-data/"
n_max = 2

sat_pattern = "msevi-medi-(.{13})\.nc"
label_pattern = "nwcsaf_msevi-medi-(.{13})\.nc"

sat_comp = re.compile(sat_pattern)
label_comp = re.compile(label_pattern)
data_sets, sat_files, lab_files = list(), {}, {}

files = os.listdir(folder)
for file in files:
    sat_id = sat_comp.match(file)
    lab_id = label_comp.match(file)
    
    if (sat_id):
        sat_files[sat_id.group(1)] = folder  + file  
    elif (lab_id):
        lab_files[lab_id.group(1)] = folder  + file

for key in sat_files.keys():
    if(key in lab_files):
        data_sets.append([sat_files[key], lab_files[key]])
        
n = 0
for ds in data_sets:
    dh.add_training_files(ds[0], ds[1])
    n += 1
    if(n >= n_max):
        break

#### Get mask indices from provided nc-file

In [3]:
import xarray as xr
mask_file = "../data/cloud-typing-data/lsm_mask_medi.nc" 
mask = xr.open_dataset(mask_file)
masked_indices = np.where(mask["land_sea_mask"])


In [4]:
trv_name = "../training_vectors_new"
filename_cl = "../classifer_1"
filename_labels = "../predicted_labels_1.nc"

dh.set_parameters(masked_indices = masked_indices)
dh.set_parameters(verbose = True)
dh.set_parameters(samples = 100, hours = [0], difference_vectors = True, original_values = True)
dh.set_parameters(nwcsaf_in_version = "auto", nwcsaf_out_version = "v2018")
dh.set_parameters(input_channels = ['bt062', 'bt073', 'bt087', 'bt097', 'bt108', 'bt120', 'bt134'], cloudtype_channel = "ct")

In [5]:
#v,l  = dh.create_training_set(masked_indices = masked_indices)


In [6]:
#dh.save_training_set(v, l, trv_name)
v,l = dh.load_training_set(trv_name)

In [7]:
ct.set_parameters(classifer_type = "Forest", feature_preselection = False, max_depth = 20)
ct.evaluate_parameters(v,l)


Correctly identified 13640 out of 16673 labels! 
Positve rate is: 0.818089


0.8180891261320699

In [8]:
import xarray as xr
labels_3 = "../data/cloud-typing-data/nwcsaf_msevi-medi-20201106_1000.nc"
dh.set_parameters(cloudtype_channel = "ct")


real_labels = xr.open_dataset(labels_3)
dh.plot_labels(labels = real_labels, hour = 0)

Longitude/Lattide variables not found!


In [9]:
data = xr.open_dataset(dh.training_sets[0][1])
#print(data.keys())
data['y']

<xarray.DataArray 'y' (y: 500)>
array([362, 363, 364, ..., 859, 860, 861], dtype=uint16)
Coordinates:
  * y        (y) uint16 362 363 364 365 366 367 368 ... 856 857 858 859 860 861
Attributes:
    long_name:      y coordinate (row) in SEVIRI image
    standard_name:  projection_y_coordinate

In [27]:
ldat = xr.open_dataset(labels_3)
print(ldat["x"])
print(ldat["y"])
ldat["lon0"]

<xarray.DataArray 'x' (x: 1300)>
array([1562, 1563, 1564, ..., 2859, 2860, 2861], dtype=uint16)
Coordinates:
  * x        (x) uint16 1562 1563 1564 1565 1566 ... 2857 2858 2859 2860 2861
Attributes:
    long_name:      x coordinate (column) in SEVIRI image
    standard_name:  projection_x_coordinate
<xarray.DataArray 'y' (y: 500)>
array([362, 363, 364, ..., 859, 860, 861], dtype=uint16)
Coordinates:
  * y        (y) uint16 362 363 364 365 366 367 368 ... 856 857 858 859 860 861
Attributes:
    long_name:      y coordinate (row) in SEVIRI image
    standard_name:  projection_y_coordinate


<xarray.DataArray 'lon0' (time: 1)>
array([0.])
Coordinates:
  * time     (time) datetime64[ns] 2020-11-06T10:00:00
Attributes:
    long_name:  MSG Subsatellite Point
    units:      degrees_east

In [25]:
c_file = "../data/cloud-typing-data/msevi_georef.nc"
georef = xr.open_dataset(c_file)

georef

<xarray.Dataset>
Dimensions:  (cols: 2776, rows: 1004)
Coordinates:
    lat      (rows, cols) float32 ...
    lon      (rows, cols) float32 ...
Dimensions without coordinates: cols, rows
Data variables:
    *empty*
Attributes:
    description:  Infrared Meteosat-SEVIRI images from the Prime Service
    title:        Meteosat-SEVIRI Brightness Temperatures
    institution:  Leibniz Institute for Tropospheric Research
    author:       Fabian Senf (senf@tropos.de)